# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# load messages dataset
messages = pd.read_csv('messages.csv')
messages.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [3]:
# load categories dataset
categories = pd.read_csv('categories.csv')
categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [4]:
# merge datasets
df = categories.merge(messages, left_on = 'id', right_on = 'id' )
df.head()

,id,categories,message,original,genre
0,2,related-1;request-0;offer-0;aid_related-0;medi...,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,related-1;request-0;offer-0;aid_related-1;medi...,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,related-1;request-0;offer-0;aid_related-0;medi...,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,related-1;request-1;offer-0;aid_related-1;medi...,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,related-1;request-0;offer-0;aid_related-0;medi...,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [5]:
# create a dataframe of the 36 individual category columns
categories_split = df.categories.str.split(";",expand=True)
categories_split = categories_split.applymap(str)
categories_split_df_list=categories_split
categories_split.columns = categories_split_df_list.iloc[0, :] 

#results_analysis_df_js_unique = np.unique(categories_split_df_list[[0:].values)
categories_split_df_list.head()



,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


In [6]:
# select the first row of the categories dataframe
row = categories_split_df_list.iloc[1,:]

#row.str.split(pat = '-', expand = True)[0].tolist()

# use this row to extract a list of new column names for categories.
# one way is to apply a lambda function that takes everything 
# up to the second to last character of each string with slicing
#row.lambda()
category_colnames = row.str.split(pat = '-', expand = True)[0].tolist()
print(category_colnames)

['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


In [7]:
# rename the columns of `categories`
categories_split_df_list.columns = category_colnames
categories_split_df_list.head()
categories_split_df_list



,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
5,related-0,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
6,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
7,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-1
8,related-0,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
9,related-1,request-1,offer-0,aid_related-1,medical_help-1,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-1


### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

In [8]:
categories_split_df_list_1 = categories_split_df_list.astype(str)


for column in categories_split_df_list_1 :
    #set each value to be the last character of the string
    categories_split_df_list_1[column] = categories_split_df_list_1[column].str.split("-", n = 1, expand = True)[1]
    
    # convert column from string to numeric
    #categories[column] = categories[column]
    
#categories
print(categories_split_df_list_1 )

      related request offer aid_related medical_help medical_products  \
0           1       0     0           0            0                0   
1           1       0     0           1            0                0   
2           1       0     0           0            0                0   
3           1       1     0           1            0                1   
4           1       0     0           0            0                0   
5           0       0     0           0            0                0   
6           1       0     0           0            0                0   
7           1       1     0           1            0                0   
8           0       0     0           0            0                0   
9           1       1     0           1            1                1   
10          1       1     0           1            1                1   
11          0       0     0           0            0                0   
12          1       1     0           1            

### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [9]:
# drop the original categories column from `df`
del df['categories']



In [10]:
print(df)

          id                                            message  \
0          2  Weather update - a cold front from Cuba that c...   
1          7            Is the Hurricane over or is it not over   
2          8                    Looking for someone but no name   
3          9  UN reports Leogane 80-90 destroyed. Only Hospi...   
4         12  says: west side of Haiti, rest of the country ...   
5         14             Information about the National Palace-   
6         15                     Storm at sacred heart of jesus   
7         16  Please, we need tents and water. We are in Sil...   
8         17    I would like to receive the messages, thank you   
9         18  I am in Croix-des-Bouquets. We have health iss...   
10        20  There's nothing to eat and water, we starving ...   
11        21  I am in Petionville. I need more information r...   
12        22  I am in Thomassin number 32, in the area named...   
13        24  Let's do it together, need food in Delma 75, i..

In [11]:
print(categories_split_df_list_1)

      related request offer aid_related medical_help medical_products  \
0           1       0     0           0            0                0   
1           1       0     0           1            0                0   
2           1       0     0           0            0                0   
3           1       1     0           1            0                1   
4           1       0     0           0            0                0   
5           0       0     0           0            0                0   
6           1       0     0           0            0                0   
7           1       1     0           1            0                0   
8           0       0     0           0            0                0   
9           1       1     0           1            1                1   
10          1       1     0           1            1                1   
11          0       0     0           0            0                0   
12          1       1     0           1            

In [12]:
# concatenate the original dataframe with the new `categories` dataframe
df1 = pd.concat([categories_split_df_list_1,df], axis = 1)
df1

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,storm,fire,earthquake,cold,other_weather,direct_report,id,message,original,genre
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,1,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,14,Information about the National Palace-,Informtion au nivaux palais nationl,direct
6,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,15,Storm at sacred heart of jesus,Cyclone Coeur sacr de jesus,direct
7,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,16,"Please, we need tents and water. We are in Sil...",Tanpri nou bezwen tant avek dlo nou zon silo m...,direct
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,17,"I would like to receive the messages, thank you",Mwen ta renmen jouin messag yo. Merci,direct
9,1,1,0,1,1,1,0,0,0,0,...,0,0,0,0,0,1,18,I am in Croix-des-Bouquets. We have health iss...,"Nou kwadebouke, nou gen pwoblem sant m yo nan ...",direct


### 6. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [13]:
# check number of duplicates
#duplicate = df[df.duplicated()] # returns duplicated values 
#dups = df1.pivot_table(index = list(df1.columns), aggfunc = 'size')
#type(dups)


In [14]:
# of duplicates
df1.count()[0] - df1.drop_duplicates().count()[0]
# of entries entries for each of the unique rows
dups = df1.pivot_table(index = list(df1.columns), aggfunc = 'size')


In [15]:
#drop duplicates 
df2 = df1.drop_duplicates()

In [16]:
# check number of duplicates
df2.count()[0] - df2.drop_duplicates().count()[0]




0

### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [17]:
engine = create_engine('sqlite:///InsertDatabaseName.db')
df.to_sql('message_ds', engine, index=False)

ValueError: Table 'message_ds' already exists.

### 8. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.